In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-6d8abed3-e66e-ac8f-ae77-0d8c7bc01921)


In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/My Drive/dataset/standford_dog_breed_dataset/images

/content/drive/My Drive/dataset/standford_dog_breed_dataset/images


In [5]:
# Download all required Libraries
from tensorflow.keras.applications import ResNet50,VGG16,MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, AveragePooling2D, Flatten, Input, Dropout
from imutils import paths
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import os
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [7]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
														rotation_range=20,
														zoom_range=0.15,
														width_shift_range=0.2,
														height_shift_range=0.2,
														shear_range=0.15,
														horizontal_flip=True,
														fill_mode="nearest"
												)

In [8]:
# Create Base model excluding Top
baseModel = VGG16(include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# loop over all layers in the base model and freeze them so they will NOT be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

58892288/58889256 [==============================] - 0s 0us/step


In [9]:
# construct the head of the model that will be placed on top of the base model
headModel = baseModel.output
headModel = GlobalAveragePooling2D()(headModel)
headModel = Dense(120, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [10]:
# Define all hyperparameters
INIT_LR = 0.0001
EPOCHS = 100
BS = 256
# Define loss, optimizer and metrics
opt = Adam(learning_rate=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [11]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [ ]:
H = model.fit( 
								aug.flow_from_directory("Images", batch_size=BS, target_size=(224, 224)),
								epochs=EPOCHS,
								verbose=1
							)

Found 20254 images belonging to 120 classes.
Epoch 1/100
80/80 [==============================] - 6918s 86s/step - loss: 15.7725 - accuracy: 0.0088
Epoch 2/100
80/80 [==============================] - 318s 4s/step - loss: 12.3473 - accuracy: 0.0151
Epoch 3/100
80/80 [==============================] - 315s 4s/step - loss: 10.6958 - accuracy: 0.0232
Epoch 4/100
80/80 [==============================] - 313s 4s/step - loss: 9.4323 - accuracy: 0.0387
Epoch 5/100
80/80 [==============================] - 307s 4s/step - loss: 8.4239 - accuracy: 0.0527
Epoch 6/100
80/80 [==============================] - 307s 4s/step - loss: 7.5605 - accuracy: 0.0698
Epoch 7/100
80/80 [==============================] - 311s 4s/step - loss: 6.9148 - accuracy: 0.0905
Epoch 8/100
80/80 [==============================] - 309s 4s/step - loss: 6.3997 - accuracy: 0.1103
Epoch 9/100
80/80 [==============================] - 303s 4s/step - loss: 5.9548 - accuracy: 0.1285
Epoch 10/100
80/80 [==============================